In [ ]:
import json
import os
import re
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, XLMRobertaForSequenceClassification
import torch.nn as nn
import torch

# Load dataset
with open(r'/kaggle/input/law-reranker/train_reranker_main.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

print(f"Tổng số samples: {len(data)}")
print(f"\nVí dụ sample đầu tiên:")
print(f"  Query: {data[0]['query'][:100]}...")
print(f"  Số positives: {len(data[0]['pos'])}")
print(f"  Số negatives: {len(data[0]['neg'])}")

Tổng số samples: 2189

Ví dụ sample đầu tiên:
  Query: Thưa luật sư tôi có đăng ký kết hôn trên pháp luật nhưng nay vợ chồng bỏ nhau theo phong tục tập quá...
  Số positives: 3
  Số negatives: 5


In [ ]:
# 1. Hàm cắt nhỏ văn bản 
def clean_and_chunk_legal_text(text):
    chunks = []
    lines = text.split('\n')
    current_chunk = []
    pattern = r'^(\d+\.|[a-z]\)|[\-\+])' 
    header = "" 
    
    for line in lines:
        line = line.strip()
        if not line: continue
        if re.match(pattern, line):
            if current_chunk:
                full_chunk = " ".join(current_chunk).strip()
                if len(full_chunk.split()) > 5: 
                    chunks.append((header + " " + full_chunk) if header else full_chunk)
            current_chunk = [line]
        else:
            if len(current_chunk) == 0 and not header: header = line
            current_chunk.append(line)
            
    if current_chunk:
        full_chunk = " ".join(current_chunk).strip()
        if len(full_chunk.split()) > 5:
            chunks.append((header + " " + full_chunk) if header else full_chunk)
    return chunks if chunks else [text[:2000]]

# 2. Load và Xử lý
input_path = "/kaggle/input/law-reranker/train_reranker_main.json"
output_dir = "/kaggle/working/dataset"
os.makedirs(output_dir, exist_ok=True)

with open(input_path, "r", encoding="utf-8") as f:
    raw_data = json.load(f)

processed_data = []
for item in raw_data:
    # Chunking cả Pos và Neg
    flat_pos = []
    for p in item['pos']: flat_pos.extend(clean_and_chunk_legal_text(p))
    
    flat_neg = []
    for n in item['neg']: flat_neg.extend(clean_and_chunk_legal_text(n))
    
    if flat_pos and flat_neg:
        processed_data.append({
            "query": item['query'],
            "pos": flat_pos,      
            "neg": flat_neg[:15]   
        })

# 3. Chia Train/Val/Test (80% Train, 10% Val, 10% Test)
train_data, temp_data = train_test_split(processed_data, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

print(f"Train: {len(train_data)} samples ({len(train_data)/len(processed_data)*100:.1f}%)")
print(f"Val: {len(val_data)} samples ({len(val_data)/len(processed_data)*100:.1f}%)")
print(f"Test: {len(test_data)} samples ({len(test_data)/len(processed_data)*100:.1f}%)")

# 4. Lưu file 
train_path = f"{output_dir}/train_reranker_final.json"
val_path = f"{output_dir}/val_reranker_final.json"
test_path = f"{output_dir}/test_reranker_final.json"

with open(train_path, "w", encoding="utf-8") as f:
    json.dump(train_data, f, ensure_ascii=False, indent=2)

with open(val_path, "w", encoding="utf-8") as f:
    json.dump(val_data, f, ensure_ascii=False, indent=2)

with open(test_path, "w", encoding="utf-8") as f:
    json.dump(test_data, f, ensure_ascii=False, indent=2)

In [ ]:
!pip uninstall -y flash-attn

In [ ]:
model_name = "BAAI/bge-reranker-v2-m3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = XLMRobertaForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=1, 
    ignore_mismatched_sizes=True 
)
model.to("cuda")

2025-11-24 16:21:56.662705: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764001316.898565      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764001316.968279      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(8194, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, o

In [ ]:
class RerankDataset(Dataset):
    def __init__(self, path):
        with open(path, "r", encoding="utf-8") as f:
            self.items = json.load(f)

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        item = self.items[idx]
        query = item["query"]
        
        # Lấy tất cả positives và negatives (không chỉ 1)
        pos_list = item["pos"] if isinstance(item["pos"], list) else [item["pos"]]
        neg_list = item["neg"] if isinstance(item["neg"], list) else [item["neg"]]
        
        # Chọn ngẫu nhiên 1 pos và 1 neg cho mỗi sample
        pos_text = np.random.choice(pos_list)
        neg_text = np.random.choice(neg_list)
        
        return query, pos_text, neg_text

def collate_fn(batch):
    queries = [b[0] for b in batch]
    pos_docs = [b[1] for b in batch]
    neg_docs = [b[2] for b in batch]

    # Tokenize pairs: [query, doc] cho cả pos và neg
    pos_pairs = [[q, d] for q, d in zip(queries, pos_docs)]
    neg_pairs = [[q, d] for q, d in zip(queries, neg_docs)]

    pos_inputs = tokenizer(pos_pairs, padding=True, truncation=True, max_length=512, return_tensors="pt")
    neg_inputs = tokenizer(neg_pairs, padding=True, truncation=True, max_length=512, return_tensors="pt")
    
    return pos_inputs, neg_inputs

dataset = RerankDataset("/kaggle/working/dataset/train_reranker_final.json") 
loader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)

for pos_batch, neg_batch in loader:
    print(pos_batch.input_ids.shape, neg_batch.input_ids.shape)
    break

torch.Size([4, 171]) torch.Size([4, 150])


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.MarginRankingLoss(margin=1.0)

## 📊 Chọn Best Model dựa trên Metric nào?

**Chọn Best Model dựa trên: `nDCG@10`** 
- Đây là metric tốt nhất cho reranking vì nó xem xét cả vị trí và thứ hạng
- Nếu nDCG@10 bằng nhau, có thể xem xét MRR@10 hoặc Accuracy@1

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Hàm đánh giá trên validation set
def evaluate_on_val(model, tokenizer, val_data_path, device="cuda", top_k=10):
    """Đánh giá model trên validation set"""
    model.eval()
    
    with open(val_data_path, 'r', encoding='utf-8') as f:
        val_data = json.load(f)
    
    mrr_scores = []
    acc_1_scores = []
    recall_k_scores = []
    ndcg_k_scores = []
    
    with torch.no_grad():
        for item in tqdm(val_data, desc="Evaluating on Val"):
            query = item['query']
            pos_doc = item['pos'][0] if isinstance(item['pos'], list) else item['pos']
            neg_docs = item['neg'] if isinstance(item['neg'], list) else [item['neg']]
            neg_docs = neg_docs[:top_k-1]
            
            candidates = [pos_doc] + neg_docs
            true_label_index = 0
            pairs = [[query, doc] for doc in candidates]
            
            inputs = tokenizer(
                pairs, 
                padding=True, 
                truncation=True, 
                max_length=512, 
                return_tensors='pt'
            ).to(device)
            
            outputs = model(**inputs)
            scores = outputs.logits.view(-1).cpu().numpy()
            
            sorted_indices = np.argsort(scores)[::-1]
            rank = np.where(sorted_indices == true_label_index)[0][0] + 1
            
            # Acc@1
            acc_1_scores.append(1 if rank == 1 else 0)
            
            # MRR
            mrr_scores.append(1 / rank if rank <= top_k else 0)
            
            # Recall@k
            recall_k_scores.append(1 if rank <= top_k else 0)
            
            # nDCG@k
            if rank <= top_k:
                dcg = 1.0 / np.log2(rank + 1)
                idcg = 1.0 / np.log2(2)
                ndcg_k_scores.append(dcg / idcg)
            else:
                ndcg_k_scores.append(0.0)
    
    model.train()
    return {
        "acc@1": np.mean(acc_1_scores),
        "mrr@10": np.mean(mrr_scores),
        "recall@k": np.mean(recall_k_scores),
        "ndcg@k": np.mean(ndcg_k_scores)
    }

# Test hàm evaluate
val_path = "/kaggle/working/dataset/val_reranker_final.json"
if os.path.exists(val_path):
    print("✅ Validation set đã được tạo")
else:
    print("⚠️  Validation set chưa có, cần chạy lại cell xử lý data")

In [ ]:
from tqdm import tqdm

model.train()
val_path = "/kaggle/working/dataset/val_reranker_final.json"

for epoch in range(4):
    total_loss = 0
    pbar = tqdm(loader, desc=f"Epoch {epoch+1}")

    for pos_inputs, neg_inputs in pbar:
        # Move to GPU
        pos_inputs = {k: v.to("cuda") for k, v in pos_inputs.items()}
        neg_inputs = {k: v.to("cuda") for k, v in neg_inputs.items()}

        # Forward pass
        pos_outputs = model(**pos_inputs)
        neg_outputs = model(**neg_inputs)
        
        # Lấy scores (logits)
        pos_scores = pos_outputs.logits.squeeze(-1)  # Shape: [batch_size]
        neg_scores = neg_outputs.logits.squeeze(-1)  # Shape: [batch_size]
        
        # Margin Ranking Loss: loss(x1, x2, y) = max(0, -y * (x1 - x2) + margin)
        # Ở đây: x1 = pos_scores, x2 = neg_scores, y = 1 (vì muốn pos > neg)
        # => loss = max(0, -(pos_scores - neg_scores) + margin)
        # => loss = max(0, neg_scores - pos_scores + margin)
        # Mục tiêu: pos_scores > neg_scores + margin
        target = torch.ones_like(pos_scores)  # y = 1 nghĩa là pos_scores nên lớn hơn neg_scores
        loss = loss_fn(pos_scores, neg_scores, target)

        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        pbar.set_postfix({"loss": f"{loss.item():.4f}"})

    avg_loss = total_loss / len(loader)
    print(f"\nEpoch {epoch+1} finished. Avg Loss: {avg_loss:.4f}")
    
    # Đánh giá trên validation set sau mỗi epoch
    if os.path.exists(val_path):
        val_metrics = evaluate_on_val(model, tokenizer, val_path, device="cuda")
        print(f"  Val Acc@1: {val_metrics['acc@1']:.4f}, Val MRR@10: {val_metrics['mrr@10']:.4f}")
        print(f"  Val Recall@10: {val_metrics['recall@k']:.4f}, Val nDCG@10: {val_metrics['ndcg@k']:.4f}")
    print("-" * 50)

Epoch 1: 100%|██████████| 493/493 [08:27<00:00,  1.03s/it, loss=0.5493]


Epoch 1 finished. Avg Loss: 0.5891579867134713


Epoch 2: 100%|██████████| 493/493 [08:29<00:00,  1.03s/it, loss=0.2641]


Epoch 2 finished. Avg Loss: 0.4202923873594337


Epoch 3: 100%|██████████| 493/493 [08:31<00:00,  1.04s/it, loss=0.0532]


Epoch 3 finished. Avg Loss: 0.2596782846305459


Epoch 4: 100%|██████████| 493/493 [08:34<00:00,  1.04s/it, loss=0.0204]

Epoch 4 finished. Avg Loss: 0.1790790672873999


In [ ]:
# Lưu model và tokenizer
output_model_dir = '/kaggle/working/output/model'
os.makedirs(output_model_dir, exist_ok=True)
model.save_pretrained(output_model_dir)
tokenizer.save_pretrained(output_model_dir)
print(f"✅ Model đã được lưu tại: {output_model_dir}")

In [ ]:
import torch
from transformers import AutoTokenizer


# model = XLMRobertaForSequenceClassification.from_pretrained("./path_to_saved_model").to("cuda")
# tokenizer = AutoTokenizer.from_pretrained("./path_to_saved_model")
model.eval()

def calculate_metrics(test_data_path, top_k=10):
    print(f"Đang đánh giá trên file: {test_data_path}")
    
    with open(test_data_path, 'r', encoding='utf-8') as f:
        test_data = json.load(f)

    mrr_scores = []
    acc_1_scores = []
    recall_k_scores = []
    ndcg_k_scores = []

    # Duyệt qua từng mẫu test
    for item in tqdm(test_data, desc="Evaluating"):
        query = item['query']
        
        # Lấy 1 Positive đại diện (nếu pos là list)
        pos_doc = item['pos'][0] if isinstance(item['pos'], list) else item['pos']
        # Lấy danh sách Negatives
        neg_docs = item['neg'] if isinstance(item['neg'], list) else [item['neg']]

        neg_docs = neg_docs[:top_k-1] 
        
        # Tạo danh sách candidates: [Positive, Neg_1, Neg_2, ..., Neg_9]
        # Vị trí thực sự của Positive là index 0
        candidates = [pos_doc] + neg_docs
        true_label_index = 0 
        
        # Tạo các cặp (Query, Doc) để model chấm điểm
        pairs = [[query, doc] for doc in candidates]
        
        # Tokenize & Predict
        with torch.no_grad():
            inputs = tokenizer(
                pairs, 
                padding=True, 
                truncation=True, 
                max_length=512, 
                return_tensors='pt'
            ).to("cuda")
            
            outputs = model(**inputs)
            scores = outputs.logits.view(-1).cpu().numpy() 
            
        sorted_indices = np.argsort(scores)[::-1]
        
        # Tìm xem true_label_index (là 0) đang nằm ở đâu trong danh sách đã xếp
        rank = np.where(sorted_indices == true_label_index)[0][0] + 1
        
        # 1. Tính Accuracy@1 
        if rank == 1:
            acc_1_scores.append(1)
        else:
            acc_1_scores.append(0)
            
        # 2. Tính MRR (1/rank)
        if rank <= top_k:
            mrr_scores.append(1 / rank)
        else:
            mrr_scores.append(0)
        
        # 3. Tính Recall@k: Nếu positive nằm trong top-k thì = 1, ngược lại = 0
        if rank <= top_k:
            recall_k_scores.append(1)
        else:
            recall_k_scores.append(0)
        
        # 4. Tính nDCG@k
        # Với 1 relevant item: nDCG@k = 1/log2(rank+1) nếu rank <= k, ngược lại = 0
        # IDCG@k = 1/log2(2) = 1 (vì relevant item ở vị trí 1 trong ranking lý tưởng)
        if rank <= top_k:
            dcg = 1.0 / np.log2(rank + 1)  # relevance = 1 cho positive
            idcg = 1.0 / np.log2(2)  # IDCG khi relevant item ở vị trí 1
            ndcg_k_scores.append(dcg / idcg)
        else:
            ndcg_k_scores.append(0.0)

    # Tổng kết
    print("\n" + "="*30)
    print("KẾT QUẢ ĐÁNH GIÁ (EVALUATION REPORT)")
    print("="*30)
    print(f"Số lượng mẫu test: {len(test_data)}")
    print(f"Top-{top_k} Candidates mix (1 Pos + {top_k-1} Negs)")
    print("-" * 30)
    print(f"✅ Accuracy@1: {np.mean(acc_1_scores):.4f}")
    print(f"✅ MRR@{top_k}   : {np.mean(mrr_scores):.4f}")
    print(f"✅ Recall@{top_k} : {np.mean(recall_k_scores):.4f}")
    print(f"✅ nDCG@{top_k}   : {np.mean(ndcg_k_scores):.4f}")
    print("="*30)

calculate_metrics("/kaggle/working/dataset/test_reranker_final.json", top_k=10)

Đang đánh giá trên file: /kaggle/working/dataset/test_reranker_final.json


Evaluating: 100%|██████████| 219/219 [01:10<00:00,  3.11it/s]


KẾT QUẢ ĐÁNH GIÁ (EVALUATION REPORT)
Số lượng mẫu test: 219
Top-10 Candidates mix (1 Pos + 9 Negs)
------------------------------
✅ Accuracy@1: 0.5388
✅ MRR@10   : 0.6727


In [ ]:
import torch
import numpy as np
import json
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# ==========================================
# CẤU HÌNH
# ==========================================
# Load trực tiếp từ HuggingFace thay vì local folder
MODEL_NAME = "BAAI/bge-reranker-v2-m3" 
TEST_DATA_PATH = "/kaggle/working/dataset/test_reranker_final.json"
TOP_K = 10 

print(f"🔄 Đang tải Base Model: {MODEL_NAME} ...")

# Load Tokenizer & Model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
model.eval()
model.to("cuda")

print(f"🚀 Bắt đầu đánh giá Base Model trên file: {TEST_DATA_PATH}")

with open(TEST_DATA_PATH, 'r', encoding='utf-8') as f:
    test_data = json.load(f)

mrr_scores = []
acc_1_scores = []
recall_k_scores = []
ndcg_k_scores = []

# Duyệt qua từng mẫu test
for item in tqdm(test_data, desc="Evaluating Base Model"):
    query = item['query']
    
    # Lấy 1 Positive + List Negatives
    pos_doc = item['pos'][0] if isinstance(item['pos'], list) else item['pos']
    neg_docs = item['neg'] if isinstance(item['neg'], list) else [item['neg']]
    
    # Lấy Top K candidates
    current_neg_docs = neg_docs[:TOP_K-1]
    candidates = [pos_doc] + current_neg_docs
    true_label_index = 0 
    
    # Tạo Input Pairs
    pairs = [[query, doc] for doc in candidates]
    
    # Dự đoán
    with torch.no_grad():
        inputs = tokenizer(
            pairs, 
            padding=True, 
            truncation=True, 
            max_length=512, 
            return_tensors='pt'
        ).to("cuda")
        
        # Base model BGE-Reranker xuất ra logits trực tiếp
        outputs = model(**inputs)
        scores = outputs.logits.view(-1).cpu().numpy()
        
    # Tính Metrics
    sorted_indices = np.argsort(scores)[::-1]
    rank = np.where(sorted_indices == true_label_index)[0][0] + 1
    
    # Acc@1
    if rank == 1:
        acc_1_scores.append(1)
    else:
        acc_1_scores.append(0)
        
    # MRR
    mrr_scores.append(1 / rank)
    
    # Recall@k: Nếu positive nằm trong top-k thì = 1, ngược lại = 0
    if rank <= TOP_K:
        recall_k_scores.append(1)
    else:
        recall_k_scores.append(0)
    
    # nDCG@k
    # Với 1 relevant item: nDCG@k = 1/log2(rank+1) nếu rank <= k, ngược lại = 0
    # IDCG@k = 1/log2(2) = 1 (vì relevant item ở vị trí 1 trong ranking lý tưởng)
    if rank <= TOP_K:
        dcg = 1.0 / np.log2(rank + 1)  # relevance = 1 cho positive
        idcg = 1.0 / np.log2(2)  # IDCG khi relevant item ở vị trí 1
        ndcg_k_scores.append(dcg / idcg)
    else:
        ndcg_k_scores.append(0.0)

# ==========================================
# KẾT QUẢ SO SÁNH
# ==========================================
print("\n" + "="*40)
print("📊 KẾT QUẢ BASE MODEL (CHƯA FINE-TUNE)")
print("="*40)
print(f"✅ Accuracy@1: {np.mean(acc_1_scores) * 100:.2f}%")
print(f"✅ MRR@{TOP_K}   : {np.mean(mrr_scores):.4f}")
print(f"✅ Recall@{TOP_K} : {np.mean(recall_k_scores):.4f}")
print(f"✅ nDCG@{TOP_K}   : {np.mean(ndcg_k_scores):.4f}")
print("="*40)

🔄 Đang tải Base Model: BAAI/bge-reranker-v2-m3 ...
🚀 Bắt đầu đánh giá Base Model trên file: /kaggle/working/dataset/test_reranker_final.json


Evaluating Base Model: 100%|██████████| 219/219 [01:09<00:00,  3.13it/s]


📊 KẾT QUẢ BASE MODEL (CHƯA FINE-TUNE)
✅ Accuracy@1: 51.14%
✅ MRR@10   : 0.6358


# Finetune BGE Reranker v2 M3 với FlagEmbedding

Dataset format: `{"query": "...", "pos": [...], "neg": [...]}`

In [ ]:
# !pip uninstall -y flash-attn

In [ ]:
# deepspeed_config = {
#     "train_micro_batch_size_per_gpu": "auto",
#     "gradient_accumulation_steps": "auto",
#     "zero_optimization": {
#         "stage": 2,
#         "allgather_partitions": True,
#         "allgather_bucket_size": 200000000,
#         "reduce_scatter": True,
#         "reduce_bucket_size": 200000000,
#         "overlap_comm": True,
#         "contiguous_gradients": True
#     },
#     "fp16": {"enabled": False},
#     "bf16": {"enabled": "auto"},
#     "gradient_clipping": "auto",
#     "wall_clock_breakdown": False
# }

# import json
# with open('stage2.json', 'w') as f:
#     json.dump(deepspeed_config, f, indent=2)

In [ ]:
# !python -m torch.distributed.run --nproc_per_node=2 \
#     -m FlagEmbedding.finetune.reranker.encoder_only.base \
#     --output_dir ./output/bge-reranker-v2-m3-finetuned \
#     --model_name_or_path BAAI/bge-reranker-v2-m3 \
#     --train_data /kaggle/working/dataset/train_reranker_final.json \
#     --learning_rate 5e-5 \
#     --num_train_epochs 1 \
#     --per_device_train_batch_size 1 \
#     --gradient_accumulation_steps 8 \
#     --dataloader_drop_last True \
#     --query_max_len 64 \
#     --passage_max_len 512 \
#     --train_group_size 8 \
#     --logging_steps 10 \
#     --save_steps 200 \
#     --save_total_limit 2 \
#     --ddp_find_unused_parameters False \
#     --deepspeed /kaggle/working/stage2.json \
#     --warmup_ratio 0.1 \
#     --bf16 \
#     --gradient_checkpointing \
#     --overwrite_output_dir \
#     --report_to none